In [2]:
#!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
#!conda install -c conda-forge lxml --yes

In [4]:
import pandas as pd
import requests

In [5]:
from bs4 import BeautifulSoup

In [6]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.text, "html.parser")

In [7]:
table_content = soup.find(class_ = "wikitable")
table_rows = table_content.find_all('tr')

columns_headers = []
res = []

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        #copying the value of Borough in the column Nighbourhood reporting the value Not assigned
        if(row[2] == 'Not assigned'):
            row[2] = row[1]
        res.append(row)
        
    #gettin the column headers drom the html table
    tables_th = tr.find_all('th')
    if tables_th:
        for th in tables_th:
            columns_headers.append(th.text.strip());

#creation of the pandas dataframe
df = pd.DataFrame(res, columns=columns_headers)

In [8]:
#deleting the rows with Borough = "Not assigned"
delete_row = df[df['Borough']=='Not assigned'].index
df = df.drop(delete_row)

In [9]:
#merging Neighbourhood with the same Postcode
df = df.groupby(['Postcode', 'Borough'], as_index=False).agg(', '.join)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3>Explaination of the code:</h3>
<ul>1) Installation of beautifulsoup4 and lxml to make it possible the webscrapping</ul>
<ul>2) Importing of the libraries beautifulsoup4, pandas and numpy</ul>
<ul>3) Identification of the class 'wikitable' on the website of wikipedia, related to the postal codes of Toronto</ul>
<ul>4) Webscrapping the wikitable class (part of html table), replacing row by row, the Neighbourhood 'Not assigned' with the value of the corrisponding Borough</ul>
<ul>5) Creation of the Pandas dataframe</ul>
<ul>6) Drop of the rows in which the value of Borough is 'Not assigned'</ul>
<ul>7) Merging of the rows with the same post code, using the comma as divider</ul>
    
<h3>Assumptions:</h3>
<ul>- The post code is always different for each Borough (which is of course not true for the Neighbourhood</ul>

In [10]:
df.shape

(103, 3)

<h1>Starting the second part</h1>

In [16]:
#using the google apikey for geocoder, singe geocoder API doewn't work at all
my_google_APIKEY = '....' #personal key hidden
url = 'https://maps.googleapis.com/maps/api/geocode/json'

In [70]:
#creation of the column Latitude and Longitude
df['Latitude'] = ''
df['Longitude'] = ''

#defining a loop with the requests and getting the coordinates
for i in range(0, df.shape[0]):
    postcode = df.loc[i, 'Postcode'];
    params = {'key': my_google_APIKEY , 'address': '{}, Toronto, Ontario'.format(postcode)}
    results_json = requests.get(url, params=params).json()
    results_json_location = results_json['results'][0]['geometry']['location']
    lat = results_json_location['lat']
    lng = results_json_location['lng']
    df.loc[i, 'Latitude'] = lat
    df.loc[i, 'Longitude'] = lng

In [74]:
 df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
